In [1]:
# Need pandas to read in data, and change column names to load properly 
import sqlite3
import pandas as pd

# Read in cleaned dfs and edit column names to snakecase using list comprehension
dfhv = pd.read_csv('cleaned_home_values.csv')
dfrc = pd.read_csv('cleaned_rent_cost.csv')
dfhv.columns = [col.split(',')[0].strip().replace(' ', '_') + '_' + col.split(',')[1].strip() if ',' in col else col for col in dfhv.columns]
dfrc.columns = [col.split(',')[0].strip().replace(' ', '_') + '_' + col.split(',')[1].strip() if ',' in col else col for col in dfrc.columns]


# Check column names & data types ("Date" not set to datetime)
print(dfrc.dtypes)


# Reset date to datetime
dfhv['Date'] = pd.to_datetime(dfhv['Date'])
dfrc['Date'] = pd.to_datetime(dfrc['Date'])

# Connecting to database to load data
conn = sqlite3.connect('hv_rc_db.db')

# Write DataFrames into SQLite database with proper data types
dfhv.to_sql('table1', conn, index=False, if_exists='replace', dtype={'Date': 'DATETIME'})
dfrc.to_sql('table2', conn, index=False, if_exists='replace', dtype={'Date': 'DATETIME'})
dfhv.to_sql('table1', conn, index=False, if_exists='replace', dtype={col: 'FLOAT' if col != 'Date' else 'DATETIME' for col in dfhv.columns})
dfrc.to_sql('table2', conn, index=False, if_exists='replace', dtype={col: 'FLOAT' if col != 'Date' else 'DATETIME' for col in dfrc.columns})

# Might be redundant, but triple checking datetime format in date column (had some issues in first draft)
# PRAGMA table_info calls the column information to check
# Run an ALTER TABLE statement if data type is incorrect
cursor = conn.cursor()
cursor.execute("PRAGMA table_info('table1');")
table1_info = cursor.fetchall()
if table1_info[0][2] != 'DATETIME':
    cursor.execute("ALTER TABLE table1 MODIFY COLUMN Date DATETIME;")
    
cursor.execute("PRAGMA table_info('table2');")
table2_info = cursor.fetchall()
if table2_info[0][2] != 'DATETIME':
    cursor.execute("ALTER TABLE table2 MODIFY COLUMN Date DATETIME;")

# Attempting to set PK on table 1 and FK on additional tables
cursor.execute("PRAGMA foreign_keys = ON;")

try:
    cursor.execute("ALTER TABLE table1 ADD PRIMARY KEY (Date);")
    cursor.execute("ALTER TABLE table2 ADD FOREIGN KEY (Date) REFERENCES table1(Date);")
    print("Foreign key constraints added successfully.")
except sqlite3.OperationalError as e:
    print("Error:", e)

# Commit & close db connection
conn.commit()
conn.close()






Date               object
Los_Angeles_CA    float64
Chicago_IL        float64
Dallas_TX         float64
Houston_TX        float64
                   ...   
Manhattan_KS      float64
Lawton_OK         float64
California_MD     float64
Cheyenne_WY       float64
Oak_Harbor_WA     float64
Length: 194, dtype: object
Error: near "PRIMARY": syntax error


In [2]:
# This cell reconnects to check that tables were successfully created
# Connect to db and create cursor to grab info
conn = sqlite3.connect('hv_rc_db.db')
cursor = conn.cursor()

# Select all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Loop that selects all rows in each table to show
for table in tables:
    table_name = table[0]
    print(f"Table: {table_name}")
    cursor.execute(f"SELECT * FROM {table_name};")
    rows = cursor.fetchall()
    for row in rows:
        print(row)
    print()

# Close cursor & connection
cursor.close()
conn.close()

Table: table1
('2015-01-31 00:00:00', 404171.5878, 513958.2964, 191213.5856, 172921.0387, 174429.271, 371339.4203, 202454.3216, 209432.0875, 167916.9208, 369722.2663, 203536.2761, 672632.1181, 278795.4405, 121083.4945, 330328.0584, 215584.3646, 463318.0122, 153032.906, 291418.7393, 255587.4814, 142643.1298, 173996.0429, 166259.8188, 157823.9169, 284315.7124, 310768.3129, 127231.5462, 143234.7706, 246868.6387, 191776.3528, 147376.8453, 147837.7954, 133619.8688, 118539.8865, 192966.4133, 215524.0821, 235661.3788, 164054.9526, 184170.7034, 137020.1429, 217337.7239, 118427.5936, 194291.8715, 139714.0181, 191667.449, 244526.5427, 209773.6732, 127313.2647, 143822.6542, 127320.86, 147913.767, 172085.5727, 552699.3688, 129561.5197, 195038.2497, 221052.2814, 147034.6545, 387453.02, 142612.4125, 167103.2235, 175737.7524, 193337.0726, 146254.7925, 168715.1422, 88637.7248, 203813.1061, 122128.1086, 180586.9412, 496960.1837, 131489.6013, 210151.2204, 209066.8666, 124140.1924, 261743.4149, 182412.15

In [3]:

# THis cell works similarly to the cell above, but this time for checking that data types are correct
# Connect to database & create cursor
conn = sqlite3.connect('hv_rc_db.db')
cursor = conn.cursor()

# Select all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Loop that calls info on all columns to see data types
for table in tables:
    table_name = table[0]
    print(f"Table: {table_name}")
    cursor.execute(f"PRAGMA table_info('{table_name}');")
    columns_info = cursor.fetchall()
    for column_info in columns_info:
        column_name = column_info[1]
        data_type = column_info[2]
        print(f"Column: {column_name}, Data Type: {data_type}")
    print()

# Close cursor & connection to db
cursor.close()
conn.close()


Table: table1
Column: Date, Data Type: DATETIME
Column: New_York_NY, Data Type: FLOAT
Column: Los_Angeles_CA, Data Type: FLOAT
Column: Chicago_IL, Data Type: FLOAT
Column: Dallas_TX, Data Type: FLOAT
Column: Houston_TX, Data Type: FLOAT
Column: Washington_DC, Data Type: FLOAT
Column: Philadelphia_PA, Data Type: FLOAT
Column: Miami_FL, Data Type: FLOAT
Column: Atlanta_GA, Data Type: FLOAT
Column: Boston_MA, Data Type: FLOAT
Column: Phoenix_AZ, Data Type: FLOAT
Column: San_Francisco_CA, Data Type: FLOAT
Column: Riverside_CA, Data Type: FLOAT
Column: Detroit_MI, Data Type: FLOAT
Column: Seattle_WA, Data Type: FLOAT
Column: Minneapolis_MN, Data Type: FLOAT
Column: San_Diego_CA, Data Type: FLOAT
Column: Tampa_FL, Data Type: FLOAT
Column: Denver_CO, Data Type: FLOAT
Column: Baltimore_MD, Data Type: FLOAT
Column: St._Louis_MO, Data Type: FLOAT
Column: Orlando_FL, Data Type: FLOAT
Column: Charlotte_NC, Data Type: FLOAT
Column: San_Antonio_TX, Data Type: FLOAT
Column: Portland_OR, Data Type: FL

In [4]:
# Original data is now stored in db, time to create new dfs to add
# Data for more home values than rental costs, want a table for home values just for cities
# that also have rental data

# Function that looks through both tables for matching column names
def get_matching_columns(database_file, table1_name, table2_name):
    # Connect to database
    conn = sqlite3.connect(database_file)

    # Create cursor
    cursor = conn.cursor()

    # Get column names for table1 (We know "Date" is a key column in each table)
    cursor.execute(f"PRAGMA table_info('{table1_name}')")
    table1_columns = [row[1] for row in cursor.fetchall() if row[1] != 'Date']

    # Get column names for table2
    cursor.execute(f"PRAGMA table_info('{table2_name}')")
    table2_columns = [row[1] for row in cursor.fetchall() if row[1] != 'Date']

    # Find column names present in both tables
    matching_columns = set(table1_columns) & set(table2_columns)

    # Close connection
    conn.close()

    return matching_columns

# Call relevant vars
database_file = 'hv_rc_db.db'
table1_name = 'table1'
table2_name = 'table2'

# matching_columns is a set of city column names that are present in both dataframes
matching_columns = get_matching_columns(database_file, table1_name, table2_name)
print("Matching column names:", matching_columns)
print(type(matching_columns))

Matching column names: {'Eugene_OR', 'Allentown_PA', 'Lynchburg_VA', 'Crestview_FL', 'Bremerton_WA', 'Phoenix_AZ', 'Rockford_IL', 'Charlottesville_VA', 'Canton_OH', 'Knoxville_TN', 'Hilton_Head_Island_SC', 'Nashville_TN', 'Greenville_NC', 'Cheyenne_WY', 'Manhattan_KS', 'Oxnard_CA', 'Columbus_GA', 'Columbus_OH', 'Modesto_CA', 'Washington_DC', 'Detroit_MI', 'Tucson_AZ', 'Fort_Wayne_IN', 'Clarksville_TN', 'Las_Cruces_NM', 'Toledo_OH', 'Chattanooga_TN', 'Gulfport_MS', 'Salinas_CA', 'Hagerstown_MD', 'East_Stroudsburg_PA', 'Lafayette_IN', 'Tulsa_OK', 'Denver_CO', 'Raleigh_NC', 'Palm_Bay_FL', 'El_Paso_TX', 'Lubbock_TX', 'Augusta_GA', 'Grand_Rapids_MI', 'Orlando_FL', 'Portland_OR', 'Merced_CA', 'Seattle_WA', 'Atlanta_GA', 'Corpus_Christi_TX', 'Harrisonburg_VA', 'Port_St._Lucie_FL', 'Fresno_CA', 'Omaha_NE', 'Las_Vegas_NV', 'Santa_Cruz_CA', 'Huntsville_AL', 'Lake_Havasu_City_AZ', 'San_Antonio_TX', 'New_Haven_CT', 'Charleston_SC', 'Atlantic_City_NJ', 'Champaign_IL', 'Columbia_MO', 'Kalamazoo_MI',

In [5]:
# Create a filtered dataframe (not currently in db) having only cities appearing in both table 1 & 2
def filter_dataframe_by_columns(dfhv, dfrc):
    # intersection is used to call common column names
    common_columns = dfhv.columns.intersection(dfrc.columns)
    
    # Filter df1 for only the common columns
    filtered_df = dfhv[common_columns]
    
    return filtered_df
filtered_df = filter_dataframe_by_columns(dfhv, dfrc)
print(filtered_df)
filtered_df.dtypes

          Date  Los_Angeles_CA   Chicago_IL    Dallas_TX   Houston_TX  \
0   2015-01-31     513958.2964  191213.5856  172921.0387  174429.2710   
1   2015-02-28     515749.3789  191520.7446  174428.1166  176102.5099   
2   2015-03-31     516131.2150  192050.9421  175918.6479  177750.8996   
3   2015-04-30     516280.3920  192570.5118  177289.4804  179226.1953   
4   2015-05-31     516921.4710  193432.5615  178724.2699  180628.5009   
..         ...             ...          ...          ...          ...   
105 2023-10-31     924447.8085  301714.5151  371738.5812  303328.2902   
106 2023-11-30     933172.3357  302609.3914  371599.7898  303044.4394   
107 2023-12-31     938650.9177  303230.2922  371273.6925  302632.6262   
108 2024-01-31     938807.2812  303876.1218  371295.3463  302626.6503   
109 2024-02-29     935800.6332  305231.3311  371776.6549  303001.5857   

     Washington_DC  Philadelphia_PA   Atlanta_GA   Phoenix_AZ  Riverside_CA  \
0      371339.4203      202454.3216  167916.

Date              datetime64[ns]
Los_Angeles_CA           float64
Chicago_IL               float64
Dallas_TX                float64
Houston_TX               float64
                       ...      
Manhattan_KS             float64
Lawton_OK                float64
California_MD            float64
Cheyenne_WY              float64
Oak_Harbor_WA            float64
Length: 187, dtype: object

In [6]:
# Supposing that a home will be a profitable rental at monthly rent of 1% of value,
# create a new_df that gives what 1% of the home values would be, with properly matched cities
# to be compared to real avg rental cost data
def calc_1_percent_df(input_df):
    # Create a new DataFrame to store 1% values
    output_df = pd.DataFrame()

    # Iterate over columns in the input DataFrame
    for col in input_df.columns:
        # Check if the column contains float values
        if input_df[col].dtype == 'float64':
            # Multiply float values by 0.01 to get 1% 
            output_df[col] = input_df[col] * 0.01
        else:
            # Copy non-float columns (Date) without conversion
            output_df[col] = input_df[col]
    
    return output_df

# Create a new DataFrame with 1% of the float values in the original DataFrame
new_df = calc_1_percent_df(filtered_df)

print(new_df)

          Date  Los_Angeles_CA   Chicago_IL    Dallas_TX   Houston_TX  \
0   2015-01-31     5139.582964  1912.135856  1729.210387  1744.292710   
1   2015-02-28     5157.493789  1915.207446  1744.281166  1761.025099   
2   2015-03-31     5161.312150  1920.509421  1759.186479  1777.508996   
3   2015-04-30     5162.803920  1925.705118  1772.894804  1792.261953   
4   2015-05-31     5169.214710  1934.325615  1787.242699  1806.285009   
..         ...             ...          ...          ...          ...   
105 2023-10-31     9244.478085  3017.145151  3717.385812  3033.282902   
106 2023-11-30     9331.723357  3026.093914  3715.997898  3030.444394   
107 2023-12-31     9386.509177  3032.302922  3712.736925  3026.326262   
108 2024-01-31     9388.072812  3038.761218  3712.953463  3026.266503   
109 2024-02-29     9358.006332  3052.313311  3717.766549  3030.015857   

     Washington_DC  Philadelphia_PA   Atlanta_GA   Phoenix_AZ  Riverside_CA  \
0      3713.394203      2024.543216  1679.16

C:\Users\trist\AppData\Local\Temp\ipykernel_28424\2391098924.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output_df[col] = input_df[col] * 0.01
C:\Users\trist\AppData\Local\Temp\ipykernel_28424\2391098924.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output_df[col] = input_df[col] * 0.01
C:\Users\trist\AppData\Local\Temp\ipykernel_28424\2391098924.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joinin

In [7]:
# Updating dtype for date column
filtered_df['Date'] = pd.to_datetime(filtered_df['Date'])
new_df['Date'] = pd.to_datetime(new_df['Date'])
new_df.dtypes

C:\Users\trist\AppData\Local\Temp\ipykernel_28424\2859314077.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Date'] = pd.to_datetime(filtered_df['Date'])


Date              datetime64[ns]
Los_Angeles_CA           float64
Chicago_IL               float64
Dallas_TX                float64
Houston_TX               float64
                       ...      
Manhattan_KS             float64
Lawton_OK                float64
California_MD            float64
Cheyenne_WY              float64
Oak_Harbor_WA            float64
Length: 187, dtype: object

In [8]:
conn = sqlite3.connect('hv_rc_db.db')

# Now adding new dfs to database using same methods as original tables
filtered_df.to_sql('table3', conn, index=False, if_exists='replace', dtype={'Date': 'DATETIME'})
new_df.to_sql('table4', conn, index=False, if_exists='replace', dtype={'Date': 'DATETIME'})
filtered_df.to_sql('table3', conn, index=False, if_exists='replace', dtype={col: 'FLOAT' if col != 'Date' else 'DATETIME' for col in filtered_df.columns})
new_df.to_sql('table4', conn, index=False, if_exists='replace', dtype={col: 'FLOAT' if col != 'Date' else 'DATETIME' for col in new_df.columns})

# Same dtype check
cursor = conn.cursor()
cursor.execute("PRAGMA table_info('table3');")
table1_info = cursor.fetchall()
if table1_info[0][2] != 'DATETIME':
    cursor.execute("ALTER TABLE table3 MODIFY COLUMN Date DATETIME;")
    
cursor.execute("PRAGMA table_info('table4');")
table2_info = cursor.fetchall()
if table2_info[0][2] != 'DATETIME':
    cursor.execute("ALTER TABLE table4 MODIFY COLUMN Date DATETIME;")

cursor.execute("PRAGMA foreign_keys = ON;")

try:
    cursor.execute("ALTER TABLE table3 ADD FOREIGN KEY (Date) REFERENCES table1(Date);")
    cursor.execute("ALTER TABLE table4 ADD FOREIGN KEY (Date) REFERENCES table1(Date);")
    print("Foreign key constraints added successfully.")
except sqlite3.OperationalError as e:
    print("Error:", e)

# Commit & close db connection
conn.commit()
conn.close()

Error: near "FOREIGN": syntax error


In [9]:
conn.close()


In [10]:
# ####### Don't Run: This code will drop the database #######
# ### Only use if db is damaged ####

# import os

# # Path to db
# db_file = 'hv_rc_db.db'

# # Look for db using path, and delete, elsewise give error
# if os.path.exists(db_file):
#     os.remove(db_file)
#     print(f"The database '{db_file}' has been deleted.")
# else:
#     print(f"The database '{db_file}' not found.")